## Prepare data

In [1]:
import torch
training_device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
training_device
import numpy as np

In [2]:
#!pip install evaluate datasets transformers accelerate==1.9.0

In [3]:
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict

df = pd.read_csv("data/acc_data.csv")
df.head()

,recordId,gpt4o_judge_score,nova_judge_score,llama3_judge_score,majority_value,agreement_percentage,writing_id,task_id,level_title,activity_instructions,student_submission
0,CALL0015056,0.0,0.0,0.0,0.0,100.000000,3e46481f-4bef-4900-b990-169da9305089,af3c3b87-9a8a-449e-b4f0-737d76275fb9,Meetings,"Your meeting is finishing, and your boss is re...","Your meeting is finishing, and your boss is re..."
1,CALL0000241,0.0,0.0,0.0,0.0,100.000000,d3ca8e74-ec0f-41a1-8bc3-030edf392772,fa1e732b-7698-4b57-85da-907fc3660ec3,2-Beginner,A new colleague is looking for a place to go f...,"From_\nToAll\nHi, there.\n\nCan you help me? I..."
2,CALL0022001,0.0,0.0,0.0,0.0,100.000000,ad6bb8ac-07c2-415e-8195-19d3791f83d3,03251188-de56-444f-8971-e68e55719bc9,Research,You work for a market research firm. You take ...,s
3,CALL0022165,0.0,0.0,0.0,0.0,100.000000,85dc4eb7-4a04-4142-aeb4-740d466bbad1,3bdca4ab-e3ee-4d24-bab1-4de708580e2e,Research,Write a summary of the report in no more than ...,G
4,CALL0000146,4.0,5.0,5.0,5.0,66.666667,57e620bf-efb2-49bf-9f36-ada83b46768d,b3285147-d9b6-42f7-9ffb-7482cd450db7,2-Beginner,Write a paragraph about how your friend stays ...,My friend stays healthy and fit by exercising ...


In [4]:
df["task_id"].value_counts()

task_id
cfb893f1-bee1-4894-99f3-43af284493a5    205
5c44f26d-2f5c-4039-bcf5-378754d2fe4e    199
bdd2883c-b71b-4fb7-a44c-a53e7bbf26f4    196
4ce13bf1-4384-44a6-9ff7-5084e02f9e2d    195
7e3ee056-1866-488e-88f3-f56af8d22613    193
                                       ... 
e5780a8d-f6ef-4218-9d34-74e064c3e413     25
178cfadd-4162-4b5b-aaee-97a1bdcd4d98     25
9cf6c0c9-e87b-4e02-84c9-11266ea51d28     25
511ccd73-9db8-45e5-8f85-10e244841ce2     23
d2196935-4153-4f5c-a1d2-dae31174ae55     19
Name: count, Length: 161, dtype: int64

In [5]:
df['text'] = (
    "Prompt Level: " + df['level_title'].astype(str) +
    " [SEP] Prompt: " + df['activity_instructions'] +
    " [SEP] Response: " + df['student_submission']
)


In [6]:
df.head()

,recordId,gpt4o_judge_score,nova_judge_score,llama3_judge_score,majority_value,agreement_percentage,writing_id,task_id,level_title,activity_instructions,student_submission,text
0,CALL0015056,0.0,0.0,0.0,0.0,100.000000,3e46481f-4bef-4900-b990-169da9305089,af3c3b87-9a8a-449e-b4f0-737d76275fb9,Meetings,"Your meeting is finishing, and your boss is re...","Your meeting is finishing, and your boss is re...",Prompt Level: Meetings [SEP] Prompt: Your meet...
1,CALL0000241,0.0,0.0,0.0,0.0,100.000000,d3ca8e74-ec0f-41a1-8bc3-030edf392772,fa1e732b-7698-4b57-85da-907fc3660ec3,2-Beginner,A new colleague is looking for a place to go f...,"From_\nToAll\nHi, there.\n\nCan you help me? I...",Prompt Level: 2-Beginner [SEP] Prompt: A new c...
2,CALL0022001,0.0,0.0,0.0,0.0,100.000000,ad6bb8ac-07c2-415e-8195-19d3791f83d3,03251188-de56-444f-8971-e68e55719bc9,Research,You work for a market research firm. You take ...,s,Prompt Level: Research [SEP] Prompt: You work ...
3,CALL0022165,0.0,0.0,0.0,0.0,100.000000,85dc4eb7-4a04-4142-aeb4-740d466bbad1,3bdca4ab-e3ee-4d24-bab1-4de708580e2e,Research,Write a summary of the report in no more than ...,G,Prompt Level: Research [SEP] Prompt: Write a s...
4,CALL0000146,4.0,5.0,5.0,5.0,66.666667,57e620bf-efb2-49bf-9f36-ada83b46768d,b3285147-d9b6-42f7-9ffb-7482cd450db7,2-Beginner,Write a paragraph about how your friend stays ...,My friend stays healthy and fit by exercising ...,Prompt Level: 2-Beginner [SEP] Prompt: Write a...


In [7]:
df = df[["text", "task_id", "level_title", "majority_value"]]
df = df.rename(columns={'majority_value': 'label'})
df.head()

,text,task_id,level_title,label
0,Prompt Level: Meetings [SEP] Prompt: Your meet...,af3c3b87-9a8a-449e-b4f0-737d76275fb9,Meetings,0.0
1,Prompt Level: 2-Beginner [SEP] Prompt: A new c...,fa1e732b-7698-4b57-85da-907fc3660ec3,2-Beginner,0.0
2,Prompt Level: Research [SEP] Prompt: You work ...,03251188-de56-444f-8971-e68e55719bc9,Research,0.0
3,Prompt Level: Research [SEP] Prompt: Write a s...,3bdca4ab-e3ee-4d24-bab1-4de708580e2e,Research,0.0
4,Prompt Level: 2-Beginner [SEP] Prompt: Write a...,b3285147-d9b6-42f7-9ffb-7482cd450db7,2-Beginner,5.0


In [8]:
# Apply the mapping to the 'labels' column
#df['label'] = df['label'].map(label_mapping)
df.dropna(subset=['label'], inplace=True)
df.reset_index(drop=True, inplace=True)

df.head()

,text,task_id,level_title,label
0,Prompt Level: Meetings [SEP] Prompt: Your meet...,af3c3b87-9a8a-449e-b4f0-737d76275fb9,Meetings,0.0
1,Prompt Level: 2-Beginner [SEP] Prompt: A new c...,fa1e732b-7698-4b57-85da-907fc3660ec3,2-Beginner,0.0
2,Prompt Level: Research [SEP] Prompt: You work ...,03251188-de56-444f-8971-e68e55719bc9,Research,0.0
3,Prompt Level: Research [SEP] Prompt: Write a s...,3bdca4ab-e3ee-4d24-bab1-4de708580e2e,Research,0.0
4,Prompt Level: 2-Beginner [SEP] Prompt: Write a...,b3285147-d9b6-42f7-9ffb-7482cd450db7,2-Beginner,5.0


In [9]:
df["label"].value_counts()

label
5.0    5733
2.0    3778
0.0    3675
3.0    3069
4.0    3010
1.0     969
Name: count, dtype: int64

In [10]:
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['text', 'task_id', 'level_title', 'label'],
    num_rows: 20234
})

In [11]:
from datasets import ClassLabel, Value, Sequence
new_features = ds.features.copy()
new_features["label"] = ClassLabel(names=[0, 1, 2, 3, 4])
ds = ds.cast(new_features)

# Step 1: Initial train/test split with stratification
train_test_ds = ds.train_test_split(test_size=0.20, seed=20)

# Step 2: Split the test set into half test, half validation
test_valid_split = train_test_ds['test'].train_test_split(test_size=0.5, seed=20)

# Step 3: Combine everything into a single DatasetDict
ds = DatasetDict({
    'train': train_test_ds['train'],
    'test': test_valid_split['train'],    # This becomes the test set
    'validation': test_valid_split['test']  # This becomes the validation set
})
ds

Casting the dataset:   0%|          | 0/20234 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'task_id', 'level_title', 'label'],
        num_rows: 16187
    })
    test: Dataset({
        features: ['text', 'task_id', 'level_title', 'label'],
        num_rows: 2023
    })
    validation: Dataset({
        features: ['text', 'task_id', 'level_title', 'label'],
        num_rows: 2024
    })
})

In [12]:
# Verify label distribution
from collections import Counter

print("Train label counts:", Counter(ds['train']['label']))
print("Test label counts:", Counter(ds['test']['label']))
print("Validation label counts:", Counter(ds['validation']['label']))

Train label counts: Counter({5: 4546, 2: 3010, 0: 2931, 3: 2481, 4: 2446, 1: 773})
Test label counts: Counter({5: 600, 2: 387, 0: 363, 3: 297, 4: 277, 1: 99})
Validation label counts: Counter({5: 587, 0: 381, 2: 381, 3: 291, 4: 287, 1: 97})


In [13]:
import json

# Fonction utilitaire pour sauvegarder un split en JSONL
def save_split_to_jsonl(dataset_split, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for record in dataset_split:
            f.write(json.dumps(record, ensure_ascii=False) + '\n')

# Sauvegarde des trois splits
save_split_to_jsonl(ds['train'], 'data/train.jsonl')
save_split_to_jsonl(ds['test'], 'data/test.jsonl')
save_split_to_jsonl(ds['validation'], 'data/validation.jsonl')

## Roberta SFT

In [14]:
import numpy as np 
import evaluate

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, cohen_kappa_score, classification_report

from scipy.stats import pearsonr

metric = evaluate.load("accuracy")

In [15]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)  # Convertir les logits en classes prédictes

    # 🎯 Exactitude (Accuracy)
    accuracy = accuracy_score(labels, predictions)

    # 🎯 Précision, Rappel et F1-score (pondérés)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")

    # 🎯 Score de Cohen's Kappa (pondéré)
    cohen_kappa = cohen_kappa_score(labels, predictions, weights="quadratic")

    # 🎯 Corrélation de Pearson
    pearson_corr, _ = pearsonr(labels, predictions)  # Retourne (coef, p-valeur), on garde seulement coef

     # 🎯 Classification Report
    class_report = classification_report(labels, predictions, output_dict=True)  # Get a dictionary of the report


    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "cohen_kappa": cohen_kappa,
        "pearson_corr": pearson_corr,
        "classification_report": class_report  # Add classification report to the return
    }


In [16]:
from datasets import load_dataset
from datasets import DatasetDict, Dataset

# Charger les fichiers JSONL en DatasetDict
dataset = DatasetDict({
    "train": load_dataset("json", data_files="data/train.jsonl")["train"],
    "test": load_dataset("json", data_files="data/test.jsonl")["train"],
    "valid": load_dataset("json", data_files="data/validation.jsonl")["train"]
})

print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'task_id', 'level_title', 'label'],
        num_rows: 16187
    })
    test: Dataset({
        features: ['text', 'task_id', 'level_title', 'label'],
        num_rows: 2023
    })
    valid: Dataset({
        features: ['text', 'task_id', 'level_title', 'label'],
        num_rows: 2024
    })
})


In [17]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-large")

In [18]:
dataset["train"][0]

{'text': 'Prompt Level: 5-Elementary [SEP] Prompt: Write about a movie you recently saw. Give facts and opinions about it. Who starred in the movie? Who directed it? What was it about? Type in the input box. Write 60-100 words. Use your own words where possible. [SEP] Response: Sweet bean\n\nIt is a movie from Japan. It was directed by Naomi Kawase. What I liked most was the photography. There were so many beautiful scenes with the trees and the old lady working together with the young man. That man runs an small "dorayaki" shop without too much interest and then an old lady appears applying for a job. That is the beginning of a very beautiful story. The plot was really original and I think it\'s a great movie to see.',
 'task_id': 'bd802cec-aaac-4a24-a654-01d84a6a972d',
 'level_title': '5-Elementary',
 'label': 4}

In [19]:
tok_test = tokenizer(dataset["train"][1]["text"], max_length=256, truncation=True)
tok_test

{'input_ids': [0, 35396, 3320, 12183, 35, 195, 12, 46331, 1766, 646, 3388, 510, 742, 42944, 35, 21062, 10, 1551, 9, 10, 2391, 147, 47, 33, 18804, 682, 4, 10391, 59, 5, 689, 6, 5, 850, 8, 932, 1493, 47, 206, 16, 505, 13, 82, 7, 216, 4, 7773, 11, 5, 8135, 2233, 4, 21062, 1191, 12, 1866, 1617, 4, 152, 1940, 16, 786, 12, 9471, 4, 646, 3388, 510, 742, 19121, 35, 229, 33850, 298, 2161, 1236, 298, 705, 4740, 267, 26144, 41460, 1717, 3252, 506, 1368, 26310, 26310, 1717, 219, 4147, 42339, 298, 725, 42339, 298, 725, 326, 19043, 8873, 8470, 14791, 3145, 14791, 3145, 1423, 506, 14791, 506, 1368, 3145, 326, 90, 428, 5967, 326, 571, 43336, 282, 385, 21978, 267, 42898, 2359, 267, 1368, 26029, 417, 1236, 1187, 344, 417, 475, 5655, 741, 21978, 4, 1437, 3296, 1910, 344, 1176, 1526, 42898, 2359, 267, 344, 1439, 1236, 267, 605, 687, 1437, 9211, 449, 330, 33130, 1717, 45882, 4112, 2359, 4112, 267, 11538, 438, 385, 330, 1071, 330, 506, 1236, 26029, 417, 1021, 14555, 10, 1910, 2628, 1176, 2628, 1910, 384, 27

In [20]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=256
    )

In [22]:
tokenized_train = dataset["train"].map(tokenize_function, batched=True)
tokenized_test = dataset["test"].map(tokenize_function, batched=True)
tokenized_valid = dataset["valid"].map(tokenize_function, batched=True)

Map:   0%|          | 0/16187 [00:00<?, ? examples/s]

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [ ]:
unique_labels = set(dataset['train']['label'])
num_labels = len(unique_labels)
num_labels

6

In [ ]:
from transformers import RobertaForSequenceClassification, TrainingArguments, Trainer
model = RobertaForSequenceClassification.from_pretrained("FacebookAI/roberta-large", num_labels=num_labels)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
args = TrainingArguments(
    output_dir="../../../model_saved/roberta-large-ft-efcamdat-augmented",
    eval_strategy="steps",  # Évaluation aux mêmes intervalles que la sauvegarde
    save_strategy="steps",  # Sauvegarde tous les 500 steps
    save_steps=200,
    eval_steps=200,  # ⚠ IMPORTANT : Évaluation aux mêmes steps
    save_total_limit=4,  # Ne garde que 4 checkpoints max
    learning_rate=2e-5,
    warmup_ratio=0.1,
    lr_scheduler_type="linear", 
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=6,
    weight_decay=0.01,
    load_best_model_at_end=True,  
    metric_for_best_model="f1",
    logging_steps=100,
    fp16=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Step,Training Loss,Validation Loss


TrainOutput(global_step=828, training_loss=0.5889229829184675, metrics={'train_runtime': 299.1896, 'train_samples_per_second': 44.28, 'train_steps_per_second': 2.767, 'total_flos': 6173196781289472.0, 'train_loss': 0.5889229829184675, 'epoch': 6.0})

In [ ]:
trainer.evaluate()

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'eval_loss': 1.1551573276519775,
 'eval_accuracy': 0.75,
 'eval_precision': 0.7535852468077532,
 'eval_recall': 0.75,
 'eval_f1': 0.7431084449411779,
 'eval_cohen_kappa': 0.9014164752905702,
 'eval_pearson_corr': 0.9070230789560548,
 'eval_classification_report': {'0': {'precision': 0.9411764705882353,
   'recall': 0.9411764705882353,
   'f1-score': 0.9411764705882353,
   'support': 34.0},
  '1': {'precision': 1.0,
   'recall': 0.5,
   'f1-score': 0.6666666666666666,
   'support': 8.0},
  '2': {'precision': 0.9111111111111111,
   'recall': 0.7192982456140351,
   'f1-score': 0.803921568627451,
   'support': 57.0},
  '3': {'precision': 0.5641025641025641,
   'recall': 0.6285714285714286,
   'f1-score': 0.5945945945945946,
   'support': 35.0},
  '4': {'precision': 0.4857142857142857,
   'recall': 0.38636363636363635,
   'f1-score': 0.43037974683544306,
   'support': 44.0},
  '5': {'precision': 0.7647058823529411,
   'recall': 0.9285714285714286,
   'f1-score': 0.8387096774193549,
   'sup

## Detailed Evaluation

In [ ]:
list_topic = dataset["valid"]["task_id"]
list_t_set = set(list_topic)
unique_t = (list(list_t_set))

list_level = dataset["valid"]["level_title"]
list_l_set = set(list_level)
unique_l = (list(list_l_set))


In [ ]:
list_r = []

# Assuming 'unique_t' is a list of unique item_ids and 'trainer' is already defined
for t in unique_t:  # Iterate over the first item in unique_t
    sub_ds = tokenized_valid.filter(lambda example: example['task_id'] == t)
    # Get predictions using the trainer
    predictions = trainer.predict(sub_ds)
    # Raw output logits (size [batch_size, num_classes])
    outputs = predictions.predictions
    # Convert logits to predicted class labels (taking the argmax across the classes)
    predicted_labels = np.argmax(outputs, axis=-1)
    ref_label = predictions.label_ids
    # Print or save the predicted classes (this will be a numpy array with the predicted class indices)
    ck = round(cohen_kappa_score(predicted_labels, ref_label, weights="quadratic"), 2)  
    pearson_corr, _ = pearsonr(ref_label, predicted_labels)
    accuracy = accuracy_score(ref_label, predicted_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(ref_label, predicted_labels, average="weighted")

    r = {
        "task_id": t,
        "level_title": sub_ds["level_title"][0],
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "ck": ck,
        "pearson": pearson_corr,
        "n_samples": len(sub_ds)
    }
    list_r.append(r)

Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

In [ ]:
list_r_level = []

# Assuming 'unique_t' is a list of unique item_ids and 'trainer' is already defined
for l in unique_l:  # Iterate over the first item in unique_t
    sub_ds = tokenized_valid.filter(lambda example: example['level_title'] == l)
    # Get predictions using the trainer
    predictions = trainer.predict(sub_ds)
    # Raw output logits (size [batch_size, num_classes])
    outputs = predictions.predictions
    # Convert logits to predicted class labels (taking the argmax across the classes)
    predicted_labels = np.argmax(outputs, axis=-1)
    ref_label = predictions.label_ids
    # Print or save the predicted classes (this will be a numpy array with the predicted class indices)
    ck = round(cohen_kappa_score(predicted_labels, ref_label, weights="quadratic"), 2)  
    pearson_corr, _ = pearsonr(ref_label, predicted_labels)
    accuracy = accuracy_score(ref_label, predicted_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(ref_label, predicted_labels, average="weighted")

    r = {
        "level_title": sub_ds["level_title"][0],
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "ck": ck,
        "pearson": pearson_corr,
        "n_samples": len(sub_ds)
    }
    list_r_level.append(r)


Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

...

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
import pandas as pd
df_eval_results = pd.DataFrame(list_r, columns=["task_id", "level_title", "accuracy", "precision", "recall", "f1", "ck", "pearson", "n_samples"])
df_eval_results.head(n=10)

,task_id,level_title,accuracy,precision,recall,f1,ck,pearson,n_samples
0,493dffa9-7513-4408-be4b-5b010a1bf051,10-Upper Intermediate,0.333333,0.407407,0.333333,0.355556,0.37,0.497863,9
1,dcdc664b-404c-407a-99a6-b9a1f99a859a,7-Intermediate,0.363636,0.545455,0.363636,0.430303,0.81,0.837415,11
2,9032f3ed-5da1-4efe-b4dd-c9470bb35d65,16-Upper Advanced,0.666667,0.633333,0.666667,0.617284,0.84,0.847094,9
3,de7a1159-0c06-49bf-a273-7bdd195d6a32,11-Upper Intermediate,0.666667,0.740741,0.666667,0.698413,0.88,0.881818,9
4,fa1e732b-7698-4b57-85da-907fc3660ec3,2-Beginner,0.333333,0.222222,0.333333,0.250000,0.86,0.965908,6
5,b4ba34ff-949d-4a82-9f56-ea472f177a71,13-Advanced,0.363636,0.200000,0.363636,0.256494,0.74,0.824022,11
6,43d154ea-bd72-4b54-9339-7c12f992b56a,15-Advanced,0.500000,0.479167,0.500000,0.469048,0.67,0.712525,8
7,23c3d59a-61d5-470f-90d1-1217b0006929,10-Upper Intermediate,1.000000,1.000000,1.000000,1.000000,1.00,1.000000,8
8,8f1a0c16-d259-4737-af06-b6b6eb9e9799,13-Advanced,0.833333,0.833333,0.833333,0.833333,0.94,0.979796,6
9,2667a44c-a31b-4b3e-a33e-144d61884196,3-Beginner,0.375000,0.687500,0.375000,0.437500,0.84,0.926179,8


In [ ]:
df_eval_results.to_csv("result_eval_data_roberta_large_writing_task_acc_sample.csv", index=False)

In [ ]:
import pandas as pd
df_eval_results_level = pd.DataFrame(list_r_level, columns=["level_title", "accuracy", "precision", "recall", "f1", "ck", "pearson", "n_samples"])
df_eval_results_level.head(n=20)

,level_title,accuracy,precision,recall,f1,ck,pearson,n_samples
0,10-Upper Intermediate,0.647059,0.666667,0.647059,0.653119,0.63,0.679126,17
1,12-Upper Intermediate,1.000000,1.000000,1.000000,1.000000,1.00,1.000000,19
2,16-Upper Advanced,0.842105,0.872180,0.842105,0.834308,0.89,0.887413,19
3,15-Advanced,0.777778,0.736953,0.777778,0.750265,0.84,0.854543,18
4,11-Upper Intermediate,0.650000,0.607143,0.650000,0.626923,0.88,0.903542,20
5,2-Beginner,0.647059,0.869748,0.647059,0.663000,0.77,0.794838,17
6,3-Beginner,0.625000,0.739583,0.625000,0.649242,0.95,0.957737,16
7,4-Elementary,0.833333,0.869444,0.833333,0.819400,0.75,0.732632,18
8,8-Intermediate,0.625000,0.504167,0.625000,0.554167,0.49,0.492524,16
9,14-Advanced,0.750000,0.900000,0.750000,0.786190,0.94,0.963761,20


In [ ]:
df_eval_results_level.to_csv("result_eval_data_roberta_large_acc_sample_by_level.csv", index=False)